# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle that contains information on 3 million used cars.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding
Rudy Russo has a problem. We've seen how bad business is. Thanks to Fuchs, his name is mud.

Something that may help Rudy is to accurately price used cars as appropriately priced cars tend to sell faster.

By leveraging existing data, we can build a supervised learning model using the sale price for predictions.
    The analysis will highlight factors influincing the price 
    The model will then identify the correlations that impact pricing
    A validated model can then be used to provide informed pricing on prospective vehicles.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

Import tools for programming

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from sklearn.experimental import enable_iterative_imputer
from sklearn import preprocessing
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

Load the dataset

In [ ]:
rudy = pd.read_csv('data/vehicles.csv')

List columns, counts and data types

In [ ]:
rudy.info()

Visualize data and headers within the table

In [ ]:
rudy.head()

Take a random sample of the table to glimpse at data distribution

In [ ]:
rudy.sample(20)

Describe the columns to look at the counts, averages and ranges of data within columns

In [ ]:
rudy.describe()

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

Drop the id and vin columns. They contain unique identifiers which are not needed for the model

In [ ]:
rudy2 = rudy.drop(columns = ['id','VIN'], axis=1)
rudy2.head()

Scrub the data to contain only lowercase and remove spaces

In [ ]:
for column in rudy2.columns[1:]:
    if rudy2[column].dtype == 'object':
        rudy2[column] = rudy2[column].str.lower().str.strip()

In [ ]:
rudy2.info()

Identify null values with columns

In [ ]:
rudy2.isnull().sum()

Fill null values with values derived from the column (impute). Two methods will be used to impute values, Extra trees regression for numarical data and Bayesian Ridge for catigorical data. To apply these methods, the data will be split by data type. Float columns are numerical and go the th eExtratreesregressor while object categories are categorical and go to the Baysesian Ridge.

In [ ]:
imputers = [
    BayesianRidge(),
    ExtraTreesRegressor(n_estimators=10, random_state=0),]

Identify numerical and categorical columns, impute values and check for remaining null values.

In [ ]:
floats = ['year', 'odometer']

object = list((Counter(rudy2.columns) -\
                    Counter(floats + ['manufacturer', 'model', 'price'])).elements())
sr_floats = rudy2[floats]
imp_floats = IterativeImputer(imputers[1])
imputed_vals = imp_floats.fit_transform(sr_floats)
rudy2[floats] = imputed_vals
rudy2.isnull().sum()[floats]

In [ ]:
def encode(data_col):
    vals = np.array(data_col.dropna())
    reshaped_data = vals.reshape(-1,1)
    encoded_data = encoder.fit_transform(reshaped_data)
    data_col.loc[data_col.notnull()] = np.squeeze(encoded_data)
    return data_col

sr_object = rudy2[object]
encoder = preprocessing.LabelEncoder()

for column in object:
    encode(sr_object[column])
    imp_categorical = IterativeImputer(BayesianRidge())
    imputed_vals_cat = imp_categorical.fit_transform(sr_object[column].values.reshape(-1, 1))
    imputed_vals_cat = imputed_vals_cat.astype('int64')
    imputed_vals_cat = pd.DataFrame(imputed_vals_cat)
    imputed_vals_cat = encoder.inverse_transform(imputed_vals_cat.values.reshape(-1, 1))
    sr_object[column] = imputed_vals_cat

rudy2[object]= sr_object
rudy2.isnull().sum()[object]

Take a look at the table header and random sample

In [ ]:
rudy2.head()

In [ ]:
rudy2.sample(10)

Since the price will be the main focus of the analysis, a glimpse of the top to prices will be helpful in understanding the distribution of prices.

In [ ]:
price_counts = rudy2.price.value_counts()
print('Price: ', price_counts.index[0], '\nCounts: ', price_counts.values[0])
print('\nTen most frequently occurring prices:\n')
print(price_counts[:10])

In [ ]:
rudy2.describe()

A histogram will help visualize the price column and distribution

In [ ]:
sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(6,3)})

def plot_histogram(col, color_val='#005c9d',\
                   x_label='Price [x10\u2076 USD]', y_label='Frequency',\
                   title_text='Distribution of car prices'):
    sns.distplot(col, kde=False, color=color_val)
    
    ax = plt.gca()
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title_text)
    ax.get_xaxis().get_major_formatter().set_scientific(False)
    ax.get_yaxis().get_major_formatter().set_scientific(False)

    plt.show()
price_mill = rudy2.price/10**6
plot_histogram(price_mill)

Some prices seem to be outlyers to the range. An histogram limiting the range to less than $75000 provides a better view.

In [ ]:
plot_histogram(rudy2.price[rudy2.price<75000])

Converting the price to a log of the price will shift the distibution more toward a normal distibution and possibly provide a better fit for the model. The log price is generated and added as a column.

In [ ]:
rudy2.insert(1, 'logprice', np.log1p(rudy2['price']))

Log of the Price plotted in a histogram

In [ ]:
plot_histogram(rudy2.logprice)

The price column is limited to the difference between the upper and lower quartile. This is known as the interquartile range (IQR). Using the IQR, outliers at the low and high ranges of the dataset are removed, allowing for analysis to be based on the middle half of the distribution and less influenced by extreme values 

In [ ]:
rudy2['price'] = rudy2.price.astype(str)
Q1 = rudy2.quantile(0.25)
Q3 = rudy2.quantile(0.75)
IQR = Q3 - Q1
rudy2 = rudy2[~((rudy2 < (Q1 - 1.5 * IQR)) | (rudy2 > (Q3 + 1.5 * IQR))).any(axis=1)]
rudy2['price'] = rudy2.price.astype(np.int64)

The model and manufacturer columns are cleaned up by dropping model names that appear less than 1000 times.  The model data is then used to fill in manufacturer null values by providing the most common manufacturer for that model.

In [ ]:
rudy2 = rudy2.groupby("model").filter(lambda x: len(x) >= 1000)
rudy2.reset_index(drop=True, inplace=True)
rudy2['manufacturer'] = rudy2.groupby('model').manufacturer.transform(
    lambda x: x.fillna(x.mode()[0]))

The manufacturers are then visualized in a count plot

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(rotation=90)
sns.countplot(rudy2.manufacturer);

Correlations between numerical columns are explored

In [ ]:
rudy2.corr()

There is a positive correlation between price and year, implying that as year increases, the price increases. This makes sense since newer cars typically cost more and observed in the plot below.

In [ ]:
year = rudy2.year.astype(np.int64)
price = rudy2.price
plt.figure(figsize=(10, 4))
plt.xticks(rotation=90)
sns.barplot(year, price)

Now we can look at the distribution of categorical data through count plots

In [ ]:
obplot = object.copy()
obplot.remove('region')
obplot.remove('state')

fig, ax = plt.subplots(3, 3, figsize=(20, 15))
for variable, subplot in zip(obplot, ax.flatten()):
    sns.countplot(rudy2[variable], ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.tight_layout()

And how the categorical data relates to price when plotted

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20, 15))
for var, subplot in zip(obplot, ax.flatten()):
    sns.barplot(x=var, y='price', data=rudy2, ax=subplot)
    for label in subplot.get_xticklabels():
        label.set_rotation(90)
plt.tight_layout()

As we proceed toward the modelling of the data, the "price" column can be dropped since the log price will be used.

In [ ]:
rudy2.drop('price', axis=1, inplace=True)

In [ ]:
col_list = ['logprice']
rearranged_cols = np.hstack((rudy2.columns.difference(col_list, sort=False), col_list))

rudy2 = rudy2.reindex(columns=rearranged_cols)

The numerical 

In [ ]:
floats

and categorical columns are listed

In [ ]:
object = list((Counter(rudy2.columns) -\
                    Counter(floats + ['logprice', 'price'])).elements())
object

In [ ]:
rudy2[object] = rudy2[object].apply(encoder.fit_transform)

And the dataframe is checked again

In [ ]:
rudy2.head()

The categorical data is transformed into numerical data to be used in the model

In [ ]:
columns_to_scale = floats + ['model']

In [ ]:
scaler = StandardScaler()

for col in columns_to_scale:
   rudy2[col] = scaler.fit_transform(np.array(rudy2[col]).reshape(-1,1))

The IQR of the log price is updated in the table

In [ ]:
Q1 = rudy2.logprice.quantile(0.25)
Q3 = rudy2.logprice.quantile(0.75)
IQR = Q3 - Q1

rudy3 = rudy2[(rudy2.logprice >= (Q1 - 1.5 * IQR)) & (rudy2.logprice <= (Q3 + 1.5 * IQR))]

Here is a sample of the table

In [ ]:
rudy3.sample(5)

A look at the plot of year and log price

In [ ]:
rudy_plot1 = sns.scatterplot(data=rudy3, x="logprice", y="year")

And a look at the scatter plot of the odometer and price

In [ ]:
rudy_plot2 = sns.scatterplot(data=rudy3, x="logprice", y="odometer")

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

The data is then split inot training and test sets in the model

In [ ]:
X = rudy3.drop(columns = 'logprice', axis = 1)
y = rudy3['logprice']
X,y

In [ ]:
train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=22)

A function to remove negative values

In [ ]:
def remove_negatives(y_test, y_pred):
    ind = [index for index in range(len(y_pred)) if(y_pred[index]>0)]
    y_pred = y_pred[ind]
    y_test = y_test[ind]
    return (y_test, y_pred)    

And a function to measure the metrics of the model are added

In [ ]:
def evaluate_perf(y_test, y_pred):
    res = []
    res.append(mean_squared_log_error(y_test, y_pred))
    res.append(np.sqrt(res[0]))
    res.append(r2_score(y_test, y_pred))
    res.append(round(r2_score(y_test, y_pred)*100, 4))
    return (res)

In [ ]:
Rudy_metrics = pd.DataFrame(index=['MSLE', 'RMSLE','R2 score','Accuracy(%)'])

A baseline look at the training and test datasets

In [ ]:
baseline_train = np.ones(shape = y_train.shape)*y_train.mean()
baseline_test = np.ones(shape = y_test.shape)*y_test.mean()
mse_baseline_train = mean_squared_error(baseline_train, y_train)
mse_baseline_test = mean_squared_error(baseline_test, y_test)
print(baseline_train.shape, baseline_test.shape)
print(f'Baseline for training data: {mse_baseline_train}')
print(f'Baseline for testing data: {mse_baseline_test}')

Now the data is fit to a linear regression model

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

And a check of the validation

In [ ]:
y_test_1, y_pred_1 = remove_negatives(y_test, y_pred)
res_lin_reg = evaluate_perf(y_test_1, y_pred_1)

print("Coefficients: \n", lin_reg.coef_)
print(f"MSLE : {res_lin_reg[0]}")
print(f"Root MSLE : {res_lin_reg[1]}")
print(f"R2 Score : {res_lin_reg[2]} or {res_lin_reg[3]}%")

Rudy_metrics["Linear"] = res_lin_reg

A plot of the actual ve predicted values

In [ ]:
df_lin_comp = pd.DataFrame({'Actual': y_test_1, 'Predicted': y_pred_1})
df_lin_comp = df_lin_comp.head(25)

df_lin_comp.plot(kind='bar', figsize=(10,5))

plt.grid(which='major', linestyle='-', linewidth='0.1', color='green')
plt.title('Linear regressor: Actual vs. predicted')
plt.ylabel('MSLE')
plt.show()

And a look at the coefficients ranked by importance.  Overall, the linear regression model does not look very good

In [ ]:
coefs = pd.Series(lin_reg.coef_, index = X_train.columns)
sorted_coefs = coefs.sort_values()

sorted_coefs.plot(kind = "barh")

plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.xlabel('Score'); 
plt.ylabel('Feature'); 
plt.title('Linear regressor: Feature importance')

plt.show()

The next model is the random forest model

In [ ]:
rando_reg = RandomForestRegressor()
n_estimators = [25, 50, 75, 100],
max_features = 0.5,
max_depth = [5, 10, 20],
min_samples_split = [1, 5, 10, 50, 100],
min_samples_leaf = [1, 5, 10, 50, 100],
bootstrap = [True, False]
rando_reg.fit(X_train,y_train)
y_pred2 = rando_reg.predict(X_test)

In [ ]:
y_test_2, y_pred_2 = remove_negatives(y_test, y_pred)
res_rf_reg = evaluate_perf(y_test_2, y_pred_2)

print(f"MSLE : {res_rf_reg[0]}")
print(f"Root MSLE : {res_rf_reg[1]}")
print(f"R2 Score : {res_rf_reg[2]} or {res_rf_reg[3]}%")

Rudy_metrics['RandomForest'] = res_rf_reg

In [ ]:
df_rf_comp = pd.DataFrame({'Actual': y_test_2, 'Predicted': y_pred_2})
df_rf_comp = df_rf_comp.head(25)

df_rf_comp.plot(kind='bar', figsize=(10,5))

plt.grid(which='major', linestyle='-', linewidth='0.1', color='green')
plt.title('Random forest: Actual vs. predicted')
plt.ylabel('MSLE')

plt.show()

In [ ]:
ridge_param_dict = {'ridge__alpha': np.logspace(0, 10, 50)}
ridge_pipe = Pipeline([('scaler', StandardScaler()), 
                      ('ridge', Ridge())])
ridge_grid = GridSearchCV(ridge_pipe, param_grid=ridge_param_dict)
ridge_grid.fit(X_train, y_train)
ridge_train_preds = ridge_grid.predict(X_train)
ridge_test_preds = ridge_grid.predict(X_test)
ridge_train_mse = mean_squared_error(y_train, ridge_train_preds)
ridge_test_mse = mean_squared_error(y_test, ridge_test_preds)
print(f'Train MSE: {ridge_train_mse}')
print(f'Test MSE: {ridge_test_mse}')
ridge_pipe

In [ ]:
ridge_pipe.fit(X_train,y_train)
y_pred3 = ridge_pipe.predict(X_test)
y_test_3, y_pred_3 = remove_negatives(y_test, y_pred)
res_ridge_reg = evaluate_perf(y_test_3, y_pred_3)

print(f"MSLE : {res_ridge_reg[0]}")
print(f"Root MSLE : {res_ridge_reg[1]}")
print(f"R2 Score : {res_ridge_reg[2]} or {res_ridge_reg[3]}%")

Rudy_metrics['Ridge'] = res_ridge_reg

In [ ]:
df_ridge_comp = pd.DataFrame({'Actual': y_test_3, 'Predicted': y_pred_3})
df_ridge_comp = df_ridge_comp.head(25)

df_ridge_comp.plot(kind='bar', figsize=(10,5))

plt.grid(which='major', linestyle='-', linewidth='0.1', color='green')
plt.title('Ridge: Actual vs. predicted')
plt.ylabel('MSLE')

plt.show()

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.